# PREPARATION

In [ ]:
import gzip, idx2numpy, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# FIX CONSTANT
CLASS = 17
THRESH = 100
MAX_EPOCHS = 50
N_SAMPLE_PLOT = 10
PICKLE_FILE = 'dataset/data.pickle'
MAP_SYMBOLS = {'+': 10, '-': 11, 
              'x': 12, '/': 13, 
              '(': 14, ')': 15, 
              ',': 16}
VEC_SYMBOLS = np.vectorize(MAP_SYMBOLS.get)

# LOAD DATA

In [ ]:
# LOAD TRAIN AND TEST DATA
data = ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']

for idx, filename in enumerate(data):
     with gzip.open('../dataset/' + filename, 'rb') as f:
        temp = f.read()
        data[idx] = idx2numpy.convert_from_string(temp)

[X_train, y_train_label, X_test, y_test_label] = data
# print Train and Test data size
print('Train:', X_train.shape, y_train_label.shape)
print('Test: ', X_test.shape, y_test_label.shape)

Each sample data is 28x28 pixel of handwritten image, convert to numpy array consist of integer between 0-255

In [ ]:
# APPEND MNIST TRAIN AND TEST DATA
X_train = np.append(X_train, X_test, axis=0)
y_train_label = np.append(y_train_label, y_test_label, axis=0)
print('Train:', X_train.shape, y_train_label.shape)

In [ ]:
def thresholding(x, thresh):
    """Masking image for simplify data.
    Convert data to binary, each pixel greater or equal to threshold will convert as 1
    """
    if x >= thresh:
        temp = 1
    else:
        temp = 0
    return temp
thresholding_v = np.vectorize(thresholding)

In [ ]:
# Masking feature data
X_train = thresholding_v(X_train, THRESH)

In [ ]:
# Reshape each sample to 1 dimension
X_train = X_train.reshape(70000, -1)

In [ ]:
# LOAD SYMBOL DATA
with open(PICKLE_FILE, 'rb') as f:
    symbol_train = pickle.load(f)

Xs_train = np.array(symbol_train['img'])
ys_train_label = VEC_SYMBOLS(np.array(symbol_train['label']))
print('Symbol:', Xs_train.shape, ys_train_label.shape)

In [ ]:
Xs_train = Xs_train.reshape(4900, -1)

In [ ]:
temp = Xs_train
temp_label = ys_train_label
for i in range(9):
    temp = np.append(temp, Xs_train, axis=0)
    temp_label = np.append(temp_label, ys_train_label, axis=0)
print(temp.shape, temp_label.shape)
Xs_train = temp
ys_train_label = temp_label

In [ ]:
X_train = np.append(X_train, Xs_train, axis=0)
y_train_label = np.append(y_train_label, ys_train_label, axis=0)

In [ ]:
print(X_train.shape, y_train_label.shape)

In [ ]:
# One hot encode label data
y_train = np.eye(CLASS)[y_train_label]

# TRAIN MODEL

In [ ]:
# CREATE MODEL 
model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(20, activation='relu'))
model.add(Dense(CLASS, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=MAX_EPOCHS, verbose=0)

In [ ]:
print ("train score: ",round(model.evaluate(X_train, y_train)[1]*100, 2) , "%")

In [ ]:
model.save('model.h5')

In [ ]:
# Plot accuracy train vs test data
plt.plot(history.history['accuracy'])